In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# función para cargar la información del syslog en un array 
# process one line and return a row
def genSyslogEntry(lines):
    etr ={} 
    etr[parm[0]] = lines[:15] #stime
    etr[parm[1]] = lines[15:].split()[0]

    ori = lines[15:].split()

    for i in range(1,len(ori)):  
        ori[i] = ori[i].replace("'",'"')
        if '=' in ori[i] != -1:
            if ori[i].find('"') != -1:
                ori[i] = ori[i].replace('"','')
                k = 1
                while (k < 10) and (i<len(ori)-1) and (ori[i+k].find('=') == -1):
                    ori[i+k] = ori[i+k].replace("'",'"')
                    ori[i] = ori[i]+' '+ori[i+k]
                    if ori[i].find('"')!=-1:
                        ori[i] = ori[i].replace('"','')
                        break
                    k = k+1
            try:
                j = parm.index(ori[i].split('=')[0])
                etr[ori[i].split('=')[0]] = ori[i].split('=')[1]
            except:
                print('[warning] line '+str(i)+' label not found '+ori[i].split('=')[0])
            
    return etr

In [3]:
# label definition
parm = 'stime ip id sn time fw pri c gcat m msg srcMac src srcZone natSrc dstMac dst dstZone natDst proto sent rcvd spkt rpkt cdur rule app n fw_action dpi appName sid appcat appid catid dstname Category arg code change note uuid op'.split()

In [4]:
# open syslog file
log = open('data/10.4.45.log','r')
lines = log.readlines()

In [5]:
# Processes line by line the syslog file
metr = []

i = 0
for line in lines:
    metr.append(genSyslogEntry(line))
    i = i+1

[warning] line 26 label not found ?ver
[warning] line 26 label not found ?ver
[warning] line 26 label not found ?ver
[warning] line 26 label not found ?ver
[warning] line 26 label not found ?ver


In [6]:
metr[26]

{'stime': 'Aug 13 06:50:09',
 'ip': '212.81.213.2',
 'id': 'firewall',
 'sn': '2CB8ED2DDA00',
 'time': '2020-08-13 06:50:09 UTC',
 'fw': '212.81.213.2',
 'pri': '1',
 'c': '0',
 'gcat': '10',
 'm': '1154',
 'srcMac': 'd8:b1:22:ed:92:f7',
 'src': '10.4.45.73:43126:X2',
 'srcZone': 'Trusted',
 'dstMac': '2c:b8:ed:2d:da:02',
 'dst': '172.217.16.227:80:X1',
 'dstZone': 'Untrusted',
 'proto': 'tcp/http',
 'rcvd': '279',
 'app': '49175',
 'appName': 'General HTTP',
 'msg': 'Application Control Detection Alert: PROTOCOLS HTTP Protocol -- GET',
 'sid': '5147',
 'appcat': 'PROTOCOLS HTTP Protocol -- GET',
 'appid': '1277',
 'catid': '74',
 'n': '577289',
 'fw_action': 'NA'}

In [7]:
# close the file
log.close()

In [8]:
# put the array in a pandas dataframe
df = pd.DataFrame(metr,columns=parm)
df.head()

,stime,ip,id,sn,time,fw,pri,c,gcat,m,...,appid,catid,dstname,Category,arg,code,change,note,uuid,op
0,Aug 13 06:29:28,212.81.213.2,firewall,2CB8ED2DDA00,2020-08-13 06:29:29 UTC,212.81.213.2,6,1024,6,537,...,1056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aug 13 06:29:29,212.81.213.2,firewall,2CB8ED2DDA00,2020-08-13 06:29:30 UTC,212.81.213.2,1,0,10,1154,...,1283,74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aug 13 06:29:29,212.81.213.2,firewall,2CB8ED2DDA00,2020-08-13 06:29:30 UTC,212.81.213.2,1,0,10,1154,...,1283,74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aug 13 06:29:29,212.81.213.2,firewall,2CB8ED2DDA00,2020-08-13 06:29:30 UTC,212.81.213.2,1,0,10,1154,...,1283,74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aug 13 06:29:29,212.81.213.2,firewall,2CB8ED2DDA00,2020-08-13 06:29:30 UTC,212.81.213.2,6,262144,6,98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# descomponemos la src Ip para tener sus partes: IP, port e if.
df['srcip'] = df['src'].apply(lambda x: x.split(':')[0])
df['srcport'] = df['src'].apply(lambda x: x.split(':')[1])
df['srcif'] = df['src'].apply(lambda x: x.split(':')[2])

In [10]:
srcip = df.groupby('srcip')

In [11]:
srcip.describe()['c']

,count,unique,top,freq
srcip,,,,
10.0.28.73,6,1,1024,6
10.0.29.153,1,1,1024,1
10.4.45.189,10921,4,1024,4825
10.4.45.73,4291,4,1024,1779
10.4.46.109,4,1,1024,4
...,...,...,...,...
8.8.8.8,11,1,0,11
93.184.220.70,1,1,0,1
95.100.117.68,7,1,0,7


In [12]:
# creamos la lista de ip que aparecen como fuente. Aquí tb están las ips que responden.reset_index para sacar la IP del índice.
srcip.count().reset_index()['srcip']

0          10.0.28.73
1         10.0.29.153
2         10.4.45.189
3          10.4.45.73
4         10.4.46.109
            ...      
136           8.8.8.8
137     93.184.220.70
138     95.100.117.68
139    95.100.244.227
140     99.80.125.208
Name: srcip, Length: 141, dtype: object

In [13]:
# Agrupamos con c=4 que es el tráfico bloqueado, la dirección origen, el nombre destino y la categoría
dstbyssrcip = df[(df['c']=='4')].groupby(['srcip','dstname', 'Category'])

In [14]:
# get url blocked (c=4) by ip
dstbyssrcip.describe()

stime  \
                                                                               count   
srcip       dstname                        Category                                    
10.4.45.189 ad.doubleclick.net             Administrative Custom List settings     6   
            bidder.criteo.com              Administrative Custom List settings    18   
            cat.nl.eu.criteo.com           Administrative Custom List settings     6   
            cm.g.doubleclick.net           Administrative Custom List settings     2   
            csm.fr.eu.criteo.net           Administrative Custom List settings     4   
            dis.criteo.com                 Administrative Custom List settings     2   
            googleads.g.doubleclick.net    Administrative Custom List settings    18   
            gum.criteo.com                 Administrative Custom List settings     8   
            pix.eu.criteo.net              Administrative Custom List settings     4   
            pubads.g.doubleclick.net       Administrative Custom List settings    15   
10.4.45.73  cdn1d-static-shared.phncdn.com Pornography                             4   
            ci.phncdn.com                  Pornography                             4   
            es.pornhub.com                 Administrative Custom List settings     4   
            fonts.playboy.com              Pornography                             6   
            googleads.g.doubleclick.net    Administrative Custom List settings    17   
            media.trafficjunky.net         Administrative Custom List settings     4   
            www.googleadservices.com       Administrative Custom List settings     2   
            www.playboy.com                Pornography                            12   
            www.pornhub.com                Administrative Custom List settings     1   

                                                                                       \
                                                                               unique   
srcip       dstname                        Category                                     
10.4.45.189 ad.doubleclick.net             Administrative Custom List settings      2   
            bidder.criteo.com              Administrative Custom List settings     10   
            cat.nl.eu.criteo.com           Administrative Custom List settings      4   
            cm.g.doubleclick.net           Administrative Custom List settings      1   
            csm.fr.eu.criteo.net           Administrative Custom List settings      2   
            dis.criteo.com                 Administrative Custom List settings      1   
            googleads.g.doubleclick.net    Administrative Custom List settings     14   
            gum.criteo.com                 Administrative Custom List settings      5   
            pix.eu.criteo.net              Administrative Custom List settings      3   
            pubads.g.doubleclick.net       Administrative Custom List settings      9   
10.4.45.73  cdn1d-static-shared.phncdn.com Pornography                              3   
            ci.phncdn.com                  Pornography                              3   
            es.pornhub.com                 Administrative Custom List settings      4   
            fonts.playboy.com              Pornography                              4   
            googleads.g.doubleclick.net    Administrative Custom List settings     15   
            media.trafficjunky.net         Administrative Custom List settings      4   
            www.googleadservices.com       Administrative Custom List settings      1   
            www.playboy.com                Pornography                              9   
            www.pornhub.com                Administrative Custom List settings      1   

                                                                                                 \
                                                                                    

In [15]:
# tenemos un informe hecho

In [16]:
# get ips of internal adresseses (X2)
internal_ip = df[(df['srcif']=='X2')]['srcip'].unique()

In [17]:
# direcciones ip internas
print(internal_ip)

['10.4.45.73' '10.4.45.189' '172.22.193.5' '10.4.46.109' '10.0.28.73'
 '10.0.29.153']


In [19]:
# export to excel
df.to_excel('10.4.45.xlsx',sheet_name='Sheet1')

In [20]:
# conversión de stime a formato de time
df['stime'] = df['stime'].apply(lambda x: datetime.strptime(df.iloc[0]['time'], '%Y-%m-%d %H:%M:%S %Z'))

In [21]:
# descomponemos la dst Ip para tener sus partes: IP, port e if.
df['dstip'] = df['dst'].apply(lambda x: x.split(':')[0])
df['dstport'] = df['dst'].apply(lambda x: x.split(':')[1])
df['dstif'] = df['dst'].apply(lambda x: x.split(':')[2])

In [22]:
df.head()

,stime,ip,id,sn,time,fw,pri,c,gcat,m,...,change,note,uuid,op,srcip,srcport,srcif,dstip,dstport,dstif
0,2020-08-13 06:29:29,212.81.213.2,firewall,2CB8ED2DDA00,2020-08-13 06:29:29 UTC,212.81.213.2,6,1024,6,537,...,NaN,NaN,NaN,NaN,10.4.45.73,44080,X2,64.233.184.188,5228,X1
1,2020-08-13 06:29:29,212.81.213.2,firewall,2CB8ED2DDA00,2020-08-13 06:29:30 UTC,212.81.213.2,1,0,10,1154,...,NaN,NaN,NaN,NaN,10.4.45.73,48982,X2,194.30.6.1,53,X1
2,2020-08-13 06:29:29,212.81.213.2,firewall,2CB8ED2DDA00,2020-08-13 06:29:30 UTC,212.81.213.2,1,0,10,1154,...,NaN,NaN,NaN,NaN,10.4.45.73,48982,X2,194.30.6.1,53,X1
3,2020-08-13 06:29:29,212.81.213.2,firewall,2CB8ED2DDA00,2020-08-13 06:29:30 UTC,212.81.213.2,1,0,10,1154,...,NaN,NaN,NaN,NaN,194.30.6.1,53,X1,10.4.45.73,48982,X2
4,2020-08-13 06:29:29,212.81.213.2,firewall,2CB8ED2DDA00,2020-08-13 06:29:30 UTC,212.81.213.2,6,262144,6,98,...,NaN,NaN,NaN,NaN,10.4.45.73,55629,X2,194.30.6.1,53,X1


In [23]:
# https://stackoverflow.com/questions/22345951/python-idiom-for-creating-dict-of-dict-of-list
# create a table with conversations
# It is a dictionary of dictionaries. First dict are internla ip adresses. Secondictionary are external adresses 
# (plus internal port and external port) in conversation with internal adresses. 
# {ip_int:{ip_ext+port_ext+port_int:[[array of lines], [array of parameter for each line]]}}
# internal ip can be src and dst, and external ip can be src and dst too.

conver = {}

for i in range(0,len(df)):
    # solo para las ips internas vamos a generar un array de conversaciones
    srcport = df.iloc[i]['srcport']
    dstip   = df.iloc[i]['dstip']
    dstport = df.iloc[i]['dstport']
    srcip   = df.iloc[i]['srcip']    
    appcat  = df.iloc[i]['appcat']
    proto   = df.iloc[i]['proto']
    sent    = df.iloc[i]['sent']
    rcvd    = df.iloc[i]['rcvd']
    cdur    = df.iloc[i]['cdur']
    app     = df.iloc[i]['app']
    n       = df.iloc[i]['n']
    sid     = df.iloc[i]['sid']
    appid   = df.iloc[i]['appid']
    catid   = df.iloc[i]['catid']
    dstname = df.iloc[i]['dstname']
    time    = df.iloc[i]['time']
    code    = df.iloc[i]['code']       # CFS custom category
    m       = df.iloc[i]['m']          # action category, summarizes gcat and c
       
    if dstport == '53' or srcport == '53':
        continue
    else:
        if srcip in internal_ip:
            # print(str(i)+'  '+srcip)
            if not srcip in conver.keys():
                #print(srcip)
                conver[srcip] = {}
            if not dstip+':'+dstport+':'+srcport in conver[srcip].keys():
                #print(dstip+':'+dstport+':'+srcport)
                conver[srcip][dstip+':'+dstport+':'+srcport] = [[i],[time, code, m, app, appcat, proto, sent, rcvd, cdur, n, sid, appid, catid, dstname]] 
            else:
                conver[srcip][dstip+':'+dstport+':'+srcport][0].append([i])
                conver[srcip][dstip+':'+dstport+':'+srcport][1].append([time, code, m, app, appcat, proto, sent, rcvd, cdur, n, sid, appid, catid, dstname]) 

        if dstip in internal_ip:
            if not dstip in conver.keys():
                #print(srcip)
                conver[dstip] = {}
            if not srcip+':'+srcport+':'+dstport in conver[dstip].keys():
                #print(dstip+':'+dstport+':'+srcport)
                conver[dstip][srcip+':'+srcport+':'+dstport] = [[i],[time, code, m, app, appcat, proto, sent, rcvd, cdur, n, sid, appid, catid, dstname]] 
            else:
                conver[dstip][srcip+':'+srcport+':'+dstport][0].append([i])
                conver[dstip][srcip+':'+srcport+':'+dstport][1].append([time, code, m, app, appcat, proto, sent, rcvd, cdur, n, sid, appid, catid, dstname]) 


In [25]:
# https://stackoverflow.com/questions/22345951/python-idiom-for-creating-dict-of-dict-of-list
# create a table with conversations
# It is a dictionary of dictionaries. First dict are internla ip adresses. Secondictionary are external adresses 
# (plus internal port and external port) in conversation with internal adresses. 
# {ip_int:{ip_ext+port_ext+port_int:[[array of lines], [array of parameter for each line]]}}
# internal ip can be src and dst, and external ip can be src and dst too.

conver = {}

for i in range(0,len(df)):
    # solo para las ips internas vamos a generar un array de conversaciones
    srcport = df.iloc[i]['srcport']
    dstip   = df.iloc[i]['dstip']
    dstport = df.iloc[i]['dstport']
    srcip   = df.iloc[i]['srcip']    
    appcat  = df.iloc[i]['appcat']
    proto   = df.iloc[i]['proto']
    sent    = df.iloc[i]['sent']
    rcvd    = df.iloc[i]['rcvd']
    cdur    = df.iloc[i]['cdur']
    app     = df.iloc[i]['app']
    n       = df.iloc[i]['n']
    sid     = df.iloc[i]['sid']
    appid   = df.iloc[i]['appid']
    catid   = df.iloc[i]['catid']
    dstname = df.iloc[i]['dstname']
    time    = df.iloc[i]['time']
    code    = df.iloc[i]['code']       # CFS custom category
    m       = df.iloc[i]['m']          # action category, summarizes gcat and c
       
    if srcip in internal_ip:
        # print(str(i)+'  '+srcip)
        if not srcip in conver.keys():
            #print(srcip)
            conver[srcip] = {}
        if not dstip+':'+dstport+':'+srcport in conver[srcip].keys():
            #print(dstip+':'+dstport+':'+srcport)
            conver[srcip][dstip+':'+dstport+':'+srcport] = [[i],[time, code, m, app, appcat, proto, sent, rcvd, cdur, n, sid, appid, catid, dstname]] 
        else:
            conver[srcip][dstip+':'+dstport+':'+srcport][0].append([i])
            conver[srcip][dstip+':'+dstport+':'+srcport][1].append([time, code, m, app, appcat, proto, sent, rcvd, cdur, n, sid, appid, catid, dstname]) 

    if dstip in internal_ip:
        if not dstip in conver.keys():
            #print(srcip)
            conver[dstip] = {}
        if not srcip+':'+srcport+':'+dstport in conver[dstip].keys():
            #print(dstip+':'+dstport+':'+srcport)
            conver[dstip][srcip+':'+srcport+':'+dstport] = [[i],[time, code, m, app, appcat, proto, sent, rcvd, cdur, n, sid, appid, catid, dstname]] 
        else:
            conver[dstip][srcip+':'+srcport+':'+dstport][0].append([i])
            conver[dstip][srcip+':'+srcport+':'+dstport][1].append([time, code, m, app, appcat, proto, sent, rcvd, cdur, n, sid, appid, catid, dstname]) 


In [24]:
conver['10.4.45.73']

{'64.233.184.188:5228:44080': [[0, [16316], [16317], [16318]],
  ['2020-08-13 06:29:29 UTC',
   nan,
   '537',
   '926',
   'IM',
   'tcp/5228',
   '2386',
   '11052',
   '1925566',
   '49063697',
   nan,
   '1056',
   nan,
   nan,
   ['2020-08-13 05:57:23 UTC',
    nan,
    '98',
    '49201',
    nan,
    'tcp/5228',
    '60',
    nan,
    nan,
    '8667099',
    nan,
    nan,
    nan,
    nan],
   ['2020-08-13 05:57:23 UTC',
    nan,
    '1154',
    '49201',
    'IM Google Talk -- SSL Activity 1',
    'tcp/5228',
    nan,
    '569',
    nan,
    '577140',
    '7083',
    '1056',
    '11',
    nan],
   ['2020-08-13 05:57:23 UTC',
    nan,
    '1154',
    '7083',
    'IM Google Talk -- Jabber Activity 3',
    'tcp/5228',
    nan,
    '569',
    nan,
    '577141',
    '926',
    '1056',
    '11',
    nan]]],
 '216.239.38.120:443:43176': [[5, [7], [10], [12]],
  ['2020-08-13 06:29:30 UTC',
   nan,
   '98',
   '49177',
   nan,
   'tcp/https',
   '60',
   nan,
   nan,
   '8668268',
   nan,

In [25]:
df['app'].unique()

array(['926', '49169', '5183', '6818', '49177', '49175', '5147', '13550',
       '11819', '7927', '49201', '7083', '7926', '4398', '13553', '6820',
       '10520', '8459', '49241', '6819', '7000', '6921', '7', '5608',
       '9808', '10521', '8360', '9380', '6754', '11873', '13872', '49330',
       '676', '351', '10637', '5', '3548', '11052', '14703', '10822',
       '8599', '10875', '976', '7841', '7898', '11680', '10253', '10938',
       '6', '5649', '8062', '13589', '2822', '967', '12596', '11646',
       '11644', '7713', '6015', '13554', '3983', '7668', '11872', '6021',
       '6011', '9807', '9335', '3618', '3097', '4395', '11241'],
      dtype=object)

In [36]:
# Read Sonicwall app control codes
appctrl = pd.read_excel('data\ID app control Sonicwall.xlsx', index_col=0)

In [37]:
appctrl

,appcat,app,appmsg,number
#,,,,
1,APP-UPDATE,360Safe,Over HTTP Proxy,5600
2,APP-UPDATE,360Safe,Update Traffic 1,1197
3,APP-UPDATE,360Safe,Update Traffic 2,1199
4,APP-UPDATE,360Safe,Update Traffic 4,1201
5,APP-UPDATE,360Safe,Update Traffic 8,6539
...,...,...,...,...
3519,WEBMAIL,Yahoo! Mail,HTTP Host mail.yahoo.co.jp [Reqs DPI-SSL],6255
3520,WEBMAIL,Yahoo! Mail,HTTP Host mail.yahoo.com [Reqs DPI-SSL],2717
3521,WEBMAIL,Yahoo! Mail,HTTP Referer mail.yahoo.com [Reqs DPI-SSL],2720


In [52]:
for i in range(0,len(df)):
    print(df.iloc[i]['app'],appctrl.iloc[1]['appcat'])
    #df.iloc[i]['appcategory'] = appctrl[(appctrl['number']==df.iloc[i]['app'])]['appcat']

926 APP-UPDATE
49169 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49175 APP-UPDATE
49177 APP-UPDATE
49175 APP-UPDATE
5147 APP-UPDATE
13550 APP-UPDATE
11819 APP-UPDATE
7927 APP-UPDATE
6818 APP-UPDATE
49201 APP-UPDATE
49201 APP-UPDATE
7083 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
926 APP-UPDATE
49169 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
49175 APP-UPDATE
49177 APP-UPDATE
49175 APP-UPDATE
5147 APP-UPDATE
49177 APP-UPDATE
13550 APP-UPDATE
7927 APP-UPDATE
11819 APP-UPDATE
6818 APP-UPDATE
49201 APP-UPDATE
49201 APP-UPDATE
7083 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
49201 APP-UPDATE
49201 APP-UPDATE
7926 APP-UPDATE
6818 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
4398 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
4

6818 APP-UPDATE
6819 APP-UPDATE
5183 APP-UPDATE
5183 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
6921 APP-UPDATE
5183 APP-UPDATE
49177 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
6818 APP-UPDATE
7 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
7 APP-UPDATE
6818 APP-UPDATE
6820 APP-UPDATE
49177 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
6820 APP-UPDATE
6820 APP-UPDATE
49177 APP-UPDATE
6820 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
7 APP-UPDATE
6818 APP-UPDATE
7 APP-UPDATE
49177 APP-UPDATE
7 APP-UPDATE
7927 APP-UPDATE
6818 APP-UPDATE
6820 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
7 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
6820 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
7 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
7 APP-UPDATE
7927 APP-UPDATE
7 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
4398 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDAT

49177 APP-UPDATE
4398 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
6820 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
7 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
4398 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
4398 APP-UPDATE
6818 APP-UPDATE
4398 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
4398 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
4398 APP-UPDATE
6818 APP-UPDATE
4398 A

7927 APP-UPDATE
7927 APP-UPDATE
10521 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
7927 APP-UPDATE
49169 APP-UPDATE
5183 APP-UPDATE
10520 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
7927 APP-UPDATE
49169 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
926 APP-UPDATE
6818 APP-UPDATE
49175 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
11819 APP-UPDATE
7927 APP-UPDATE
6818 APP-UPDATE
49201 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
49201 APP-UPDATE
49201 APP-UPDATE
7927 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
49201 APP-UPDATE
7927 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
6 APP-UPDATE
6818 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
5608 APP-UPDATE
9808 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
7 APP-UPDATE
7927 AP

7927 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
10822 APP-UPDATE
7927 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
5 APP-UPDATE
7927 APP-UPDATE
6818 APP-UPDATE
7927 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
49201 APP-UPDATE
5183 APP-UPDATE
8062 APP-UPDATE
6818 APP-UPDATE
4398 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
10822 APP-UPDATE
7927 APP-UPDATE
5 APP-UPDATE
7927 APP-UPDATE
4398 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
926 APP-UPDATE
6818 APP-UPDATE
49175 APP-UPDATE
49177 APP-UPDATE
49175 APP-UPDATE
5147 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
11819 APP-UPDATE
6818 APP-UPDATE
49201 APP-UPDATE
49201 APP-UPDATE
7083 APP-UPDATE
7927 APP-UPDATE
49201 APP-UPDATE
49201 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
8360 APP-UP

...

6 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
7 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49169 APP-UPDATE
5183 APP-UPDATE
5649 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
7927 APP-UPDATE
8360 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
5 APP-UPDATE
49177 APP-UPDATE
976 APP-UPDATE
976 APP-UPDATE
976 APP-UPDATE
976 APP-UPDATE
6818 APP-UPDATE
5183 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
976 APP-UPDATE
5 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
49201 APP-UPDATE
7 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
5608 APP-UPDATE
9808 APP-UPDATE
5 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
6818 APP-UPDATE
6820 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
10521 APP-UPDATE
6818 APP-UPDATE
7 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
7 APP-UPDATE
6818 APP-UPDATE
7 APP-UPDATE
6818 AP

6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6820 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
7 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
11680 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
7927 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
7 APP-UPDATE
7927 APP-UPDATE
7 APP-UPDATE
7927 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
7927 APP-UPDATE
7 APP-UPDATE
11680 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
7927 APP-UPDATE
5183 A

6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49201 APP-UPDATE
49201 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
49169 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
8360 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
4398 APP-UPDATE
4398 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
976 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
976 APP-UPDATE
976 APP-UPDATE
976 APP-UPDATE
976 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
5 APP-UPDATE
5 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
10822 APP

49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
10822 APP-UPDATE
7927 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
5183 APP-UPDATE
6820 APP-UPDATE
49177 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
7 APP-UPDATE
49177 APP-UPDATE
49169 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
7 APP-UPDATE
6818 APP-UPDATE
6921 APP-UPDATE
4398 APP-UPDATE
4398 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
7927 APP-UPDATE
49201 APP-UPDATE
6820 APP-UPDATE
7927 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
7 APP-UPDATE
7927 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
6820 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
4917

6818 APP-UPDATE
49201 APP-UPDATE
49201 APP-UPDATE
7083 APP-UPDATE
49201 APP-UPDATE
49201 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
7927 APP-UPDATE
8360 APP-UPDATE
49177 APP-UPDATE
976 APP-UPDATE
7927 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
976 APP-UPDATE
5 APP-UPDATE
7927 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
7927 APP-UPDATE
7927 APP-UPDATE
5183 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
7927 APP-UPDATE
49169 APP-UPDATE
5183 APP-UPDATE
10520 APP-UPDATE
6818 APP-UPDATE
6818 APP-UPDATE
8599 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
6818 APP-UPDATE
49169 APP-UPDATE
5183 APP-UPDATE
6921 APP-UPDATE
6921 APP-UPDATE
6818 APP-UPDATE
49177 APP-UPDATE
49177 APP-UPDATE
6921 APP-UPDATE
6818 APP-UPDATE
5183 APP-UP

In [45]:
appctrl

KeyError: 'appcategory'

In [46]:
df.head()

,stime,ip,id,sn,time,fw,pri,c,gcat,m,...,change,note,uuid,op,srcip,srcport,srcif,dstip,dstport,dstif
0,2020-08-13 06:29:29,212.81.213.2,firewall,2CB8ED2DDA00,2020-08-13 06:29:29 UTC,212.81.213.2,6,1024,6,537,...,NaN,NaN,NaN,NaN,10.4.45.73,44080,X2,64.233.184.188,5228,X1
1,2020-08-13 06:29:29,212.81.213.2,firewall,2CB8ED2DDA00,2020-08-13 06:29:30 UTC,212.81.213.2,1,0,10,1154,...,NaN,NaN,NaN,NaN,10.4.45.73,48982,X2,194.30.6.1,53,X1
2,2020-08-13 06:29:29,212.81.213.2,firewall,2CB8ED2DDA00,2020-08-13 06:29:30 UTC,212.81.213.2,1,0,10,1154,...,NaN,NaN,NaN,NaN,10.4.45.73,48982,X2,194.30.6.1,53,X1
3,2020-08-13 06:29:29,212.81.213.2,firewall,2CB8ED2DDA00,2020-08-13 06:29:30 UTC,212.81.213.2,1,0,10,1154,...,NaN,NaN,NaN,NaN,194.30.6.1,53,X1,10.4.45.73,48982,X2
4,2020-08-13 06:29:29,212.81.213.2,firewall,2CB8ED2DDA00,2020-08-13 06:29:30 UTC,212.81.213.2,6,262144,6,98,...,NaN,NaN,NaN,NaN,10.4.45.73,55629,X2,194.30.6.1,53,X1
